In [18]:

import mujoco
import numpy as np
import mediapy as media

# Load the modified XML model
xml = """
<mujoco model="hemisphere_rat">
    <compiler angle="degree" coordinate="local"/>
    <option timestep="0.01" gravity="0 0 -9.81"/>

    <visual>
        <global offwidth="800" offheight="600"/>
    </visual>

    <worldbody>
        <!-- Hemisphere (approximated with a sphere) -->
        <body name="hemisphere" pos="0 0 0">
            <geom type="sphere" size="1" rgba="0.7 0.7 0.7 0.1" contype="1" conaffinity="1"/>
        </body>

        <!-- Rat agent (dynamic body) -->
        <body name="rat" pos="0 0 -0.95">
            <freejoint name="rat_free"/>
            <geom type="sphere" size="0.05" mass="0.1" friction="1 0.005 0.0001" rgba="1 1 1 1"/>
        </body>

        <!-- Light source -->
        <light name="light0" pos="0 0 3" dir="0 0 -1" diffuse="1 1 1" specular="0.3 0.3 0.3"/>
    </worldbody>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# Simulation parameters
duration = 10  # simulation duration in seconds
time_step = model.opt.timestep
num_steps = int(duration / time_step)

# Desired speed and mass
desired_speed = 0.1
mass = model.body('rat').mass

# Set up the camera
camera = mujoco.MjvCamera()
camera.lookat = np.array([0, 0, 0])
camera.distance = 2.5
camera.elevation = -20
camera.azimuth = 90

# Get the index of the rat body
rat_body_id = model.body('rat').id

frames = []

def get_normal(position):
    return position / np.linalg.norm(position)

def sample_random_direction(normal):
    theta = np.random.uniform(0, 2 * np.pi)
    if np.abs(normal[2]) < 1.0:
        tangent = np.array([-normal[1], normal[0], 0])
    else:
        tangent = np.array([0, -normal[2], normal[1]])
    tangent /= np.linalg.norm(tangent)
    bitangent = np.cross(normal, tangent)
    direction = np.cos(theta) * tangent + np.sin(theta) * bitangent
    return direction

# Use a context manager for the renderer
with mujoco.Renderer(model, width=800, height=600) as renderer:
    for step in range(num_steps):
        # Current position of the rat
        rat_pos = data.xpos[rat_body_id].copy()

        # Velocity in world coordinates
        vel = np.zeros(6)
        mujoco.mj_objectVelocity(model, data, mujoco.mjtObj.mjOBJ_BODY, rat_body_id, vel, flg_local=0)
        rat_vel = vel[:3]

        # Compute normal at current position
        normal = get_normal(rat_pos)

        # Sample random direction tangent to the surface
        direction = sample_random_direction(normal)

        # Desired velocity
        desired_velocity = desired_speed * direction

        # Compute acceleration and force
        acceleration = (desired_velocity - rat_vel) / time_step
        force = mass * acceleration

        # Remove the normal component to keep the force tangent to the surface
        force -= np.dot(force, normal) * normal

        # Apply the force to the rat
        data.xfrc_applied[rat_body_id, :3] = force

        # Step the simulation
        mujoco.mj_step(model, data)

        # Clear the applied force
        data.xfrc_applied[rat_body_id, :] = 0

        # Update the scene and render
        renderer.update_scene(data, camera=camera)
        frame = renderer.render()
        frames.append(frame.copy())

# Display the video
media.show_video(frames, fps=60)


C:\Users\Muhammad Sahal\AppData\Local\Temp\ipykernel_29996\3478213932.py:58: RuntimeWarning: invalid value encountered in divide
  return position / np.linalg.norm(position)


This browser does not support the video tag.
